# Lab 01 - Tokenisation and basic feature vectors

This lab is an introduction to some basic test processing that will give us the first impression of the challenges in translating text into meaningful feature vector representations.

So let's first set some text...

In [ ]:
sentence = """ Welcome to the Natural Language Processing lab!
               We'll learn many things in this no 1 lab, so we will take it easy.
               Natural Language Processing is fun."""

Let's also download some libraries we'll be using:

In [ ]:
%pip install numpy pandas sklearn nltk

## Using native Python functions for tokenisation

Let's try to split the text based on spaces using the built in string function `split()`

In [ ]:
sentence.split()

You will observe that some words are not well separated from punctuation and contain some appended onto them.
So we need to find a way to remove those characters... but, before we do that, let's see how we can create a quick feature vector first!

In [ ]:
tokens = sentence.split()  # splitting based on spaces
vocab = sorted(set(tokens))  # sorting and removing duplicates by using set()
vocab  # just printing the vocab so we can look at it

We can see that the sorted list has the numbers first, followed by capital and then lower case letters (all alphabetically sorted). We also see that repeating words appear only once in our vocabulary list. Let's compare the size of the two lists.

In [ ]:
tokens_len = len(tokens)
vocab_len = len(vocab)

print(f"Tokens: {tokens_len}")
print(f"Vocab: {vocab_len}")

Let's try and print the matrix of tokens against vocabulary. We will use the numpy lib for that.

In [ ]:
import numpy as np

matrix = np.zeros((tokens_len, vocab_len), int)
for i, token in enumerate(tokens):
    matrix[i, vocab.index(token)] = 1

matrix

It's not easy to see, but the second, third and fourth columns have the value of 1 in two rows, whereas the rest only take the value of 1 in a single row. To make it a little more readable, we could use Pandas and DataFrame! Both Pandas and NumPy are very useful libs that we will use many times.

In [ ]:
import pandas as pd

pd.DataFrame(matrix, columns=vocab)

Now this is a lot more clear and if we wanted we could carry on making it look nicer.

Let's now carry on building the bag of words (BoW)

In [ ]:
bow = {token: 1 for token in tokens}  # setting this up as a dictionary
sorted(bow.items())  # lets print it

Since bow is a dictionary, we see that there are no duplicate words.

Pandas also has a more efficient form of a dictionary called `Series`.

In [ ]:
df = pd.DataFrame(pd.Series(dict([(token, 1) for token in tokens])), columns=["sent"]).T
df

In [ ]:
corpus = {}
for i, sent in enumerate(sentence.split('\n')):
    corpus[f"sent{i}"] = dict((tok, 1) for tok in sent.split())

df = pd.DataFrame.from_records(corpus).fillna(0).astype(int).T
df

Now we see how we managed to build feature vectors for the three sentences we originally had. Now let's do a Dot Product calculation.

In [ ]:
df = df.T
print(f"Dot product of sent0 from sent1: {df.sent0.dot(df.sent1)} and dot product of sent0 from sent1: {df.sent0.dot(df.sent2)}")

As we see from the results, the higher the dot product to more similar the vectors are... so given that only the first and last sentence have some common words, we see that this comes back as 3, where as the two sentences who have nothing in common come bak as 0.

We can improve our vocabulary now if we were to remove all other punctuation. Let's do that with regular expressions.

In [ ]:
import re

tokens = re.split(r"[-\s.,;!?]+", sentence)
tokens

## NLTK

Although this seems to be great... you might still have issues with different characters that are not anticipated. So we usually use an existing NLP related tokeniser to do this job. Let's try the NLTK lib.

NLTK also supports regular expressions:

In [ ]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r"\w+|$[0-9.]+|\S+")
tokenizer.tokenize(sentence)

but there are other more specialised tokenisers:

In [ ]:
from nltk.tokenize import TreebankWordTokenizer

tokenizer = TreebankWordTokenizer()
tokenizer.tokenize(sentence)

For now let's use the regular expression special word pattern `\w`, so we can control what the tokeniser does.

In [ ]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(sentence)
print(tokens)



At the point you could try out different other tokenisers from other libraries and see if there are any differences.

We will now calculate the 2-grams:

In [ ]:
from nltk.util import ngrams

list(ngrams(tokens, 2))

and 3-grams:

In [ ]:
list(ngrams(tokens, 3))

If we want to include the n-grams as strings rather than tuples, then we need to convert them:

In [ ]:
bigrams = [" ".join(x) for x in list(ngrams(tokens, 2))]
print(f"Bigrams: {bigrams}\n")

trigrams = [" ".join(x) for x in list(ngrams(tokens, 3))]
print(f"Trigrams: {trigrams}")

Another important step we looked at in the lectures are the stop words. Let's try to use the nltk stop word list to remove them.

First, let's download the list.

In [ ]:
import nltk
nltk.download('stopwords')

and now check it:

In [ ]:
stop_words = nltk.corpus.stopwords.words('english')
print(f"number of stopwords: {len(stop_words)}")
print(stop_words)

Other libs have different stopwords. Let's see a much larger set from sklearn.

In [ ]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as sklearn_stop_words

print(f"number of stopwords: {len(sklearn_stop_words)}")
print(sklearn_stop_words)

Strangely enough, although there are more stop words in sklearn, you will find that nltk has words that are not contained in sklearn. So you might want to join the two lists.

For normalising the text you could do something as simple as making sure all words are lower case.

In [ ]:
norm_tokens = [x.lower() for x in tokens]
print(norm_tokens)

For stemming the words we could use nltk again.

In [ ]:
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()
stem_tokens = [stemmer.stem(x) for x in norm_tokens]
print(stem_tokens)

For lemmatising, nltk also does the trick.

In [ ]:
from nltk.stem import WordNetLemmatizer

nltk.download("wordnet")
nltk.download("omw-1.4")

lemmatizer = WordNetLemmatizer()
stem_tokens = [lemmatizer.lemmatize(x) for x in norm_tokens]
print(stem_tokens)

With this example sentence, there are no issues with the lemmatisation... but let's look at the following example:

In [ ]:
print(lemmatizer.lemmatize("better"))
print(lemmatizer.lemmatize("better", "a"))  # declaring the POS as adjective

If we don't include the Part-of-speech (POS), nltk, using wordnet, does not work well. So let's try to fix that.

In [ ]:
from nltk.corpus import wordnet

def get_wordnet_pos(word):
    """Map the POS tag to the first character lemmatize() accepts."""

    try:  # download nltk's POS tagger if it doesn't exist
        nltk.data.find("taggers/averaged_perceptron_tagger")
    except LookupError:
        nltk.download("averaged_perceptron_tagger")
    tag = nltk.pos_tag([word])[0][1][0].upper()  # use ntlk's POS tagger on the word

    # now we need to convert from nltk to wordnet POS notations (for compatibility reasons)
    tag_dict = {
        "J": wordnet.ADJ,
        "N": wordnet.NOUN,
        "V": wordnet.VERB,
        "R": wordnet.ADV
    }

    return tag_dict.get(tag, wordnet.NOUN)  # return and default to noun if not found

In [ ]:

stem_tokens = [lemmatizer.lemmatize(x, pos=get_wordnet_pos(x)) for x in norm_tokens]
print(stem_tokens)

If we look at the words now we are getting more counts for our bag of words.

In [ ]:
from collections import Counter

bow = Counter(stem_tokens)
bow

Now let's check the most frequent 6 words.

In [ ]:
bow.most_common(6)

Now let's remove the stop words and check the count again.

In [ ]:
no_stop_tokens = [x for x in stem_tokens if x not in stop_words]
count = Counter(no_stop_tokens)
count

Finally... let's make our feature vector using the frequency ratio (term count / total number of terms in the doc):

In [ ]:
document_vector = []
doc_length = len(no_stop_tokens)
for key, value in count.most_common():
    document_vector.append(value / doc_length)

print(document_vector)

We have explored many many options already and we will continue with more advanced feature vectors in the next lab, plus some visualisations in charts. So until then please try different experiments on your own:
1. See if you can change the text and have more sentences with different topics (so you can compare the feature vectors later)
2. Try to use different libraries for tokenising, PoS tagging, stemming and lemmatising.
3. Try to use distance metrics to compare vectors, such as Euclidian distance.